In [1]:
import pandas as pd
import numpy as np
import pyodbc
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import gensim
from collections import defaultdict
from gensim import corpora
from gensim import models
from gensim import similarities

In [ ]:
host = r''
dbSQL = ''
user = ''
pwd = ''
MC0 = '''ABS AH- ALM BC- BL- CA- CAE CHG COM CU- FP- FW- GA- GBL GPP GUE HVC LBU LGR LIT MWT PLT PND PRD PWS RS- SCR SD- SDR SPP STB SW- TIC TRF UDT WST'''.split(' ')
MC1 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC2 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- BCW SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
MC3 = '''ABS AF- AH- ALM BC- BL- CA- CAE CBA CCW CHG CND COM CU- ESP EXC FAT FG- FP- FW- GA- GUE GYP HVC LIT MWT PFC PJF PLT PRD RS- SCR SD- SDR SIS STB SW- TA- TIC TRF'''.split(' ')
#Consider adding MC00X in fron to all above in code os I don't need to do it later in teh code :)
#I will do this in an algorithm later by pulling in MaximoDW.dbo.Locations and grabbing DISTINCT SUBSTRING(LOCATION, 4, 8) but right now i'm lazy
where = ''
for l in MC0: where += f'OR LOCATION LIKE \'MC000{l}%\' '
for l in MC1: where += f'OR LOCATION LIKE \'MC001{l}%\' '
for l in MC2: where += f'OR LOCATION LIKE \'MC002{l}%\' '
for l in MC3: where += f'OR LOCATION LIKE \'MC003{l}%\' '
where = where[3:]
maximoSelect = f"""SELECT [

"""
woSelect= f"""SELECT [WONUM]
      ,[LOCATION]
      ,[DESCRIPTION]
  FROM [Generation].[agent].[MaximoWorkOrders]
  Where cast(STATUSDATE as date) > '01/01/2021'"""
 # And ({where})
#getting data and then relableing maximo location to higher level equipment name
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+host+';DATABASE='+dbSQL+';UID='+user+';PWD='+ pwd)
WOdata = pd.read_sql(woSelect, cnxn)
#for key in MC0: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC000' + key)] = 'MC000' + MC0[MC0.index(key)]
#for key in MC1: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC001' + key)] = 'MC001' + MC1[MC1.index(key)]
#for key in MC2: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC002' + key)] = 'MC002' + MC2[MC2.index(key)]
#for key in MC3: WOdata.LOCATION[WOdata.LOCATION.str.startswith('MC003' + key)] = 'MC003' + MC3[MC3.index(key)]
WOdata = WOdata.drop(columns=['WONUM'])
WOdata

In [ ]:
WOdata['Edited1'] = [list() for x in range(len(WOdata.index))]
WOdata['Edited2'] = [list() for x in range(len(WOdata.index))]

def processText(text):
    #text_tokens = word_tokenize(text)
    text_tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(text)
    text_tokens = [word for word in text_tokens if not word in stop_words]  ##Remove Stop Words
    text_tokens = [word.lower() for word in text_tokens]
    #text_tokens = [ps.stem(word) for word in text_tokens] #Stem Words
    #text_tokens = [word for word in text_tokens if len(word) > 2] #Removing noise
    #text_tokens = [word for word in text_tokens if word.isalpha()] #Remove Punctuation
    #text_tokens = [ele for ele in text_tokens if ele.strip()] #remove empty spaces in list
    return text_tokens
#Applying Text Processing
WOdata['Edited1'] = WOdata['DESCRIPTION'].apply(processText)
WOdata

In [4]:
texts = WOdata['Edited1'].to_list()
frequency = defaultdict(int)
'''
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]
'''
dictionary = corpora.Dictionary(WOdata['Edited1'])
#dictionary.filer_extremes()
corpus = [dictionary.doc2bow(text) for text in WOdata['Edited1']]


In [5]:
tfidf = models.TfidfModel(corpus, dictionary)

In [6]:
input = 'abs'
vec_bow = dictionary.doc2bow(input.lower().split(' '))
vec_lsi = tfidf[vec_bow]  # convert the query to LSI space
#print(vec_lsi)
#index = similarities.MatrixSimilarity(corpus) 
index = similarities.MatrixSimilarity(tfidf[corpus])
sims = index[vec_lsi]  # perform a similarity query against the corpus

#print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

sims = sorted(enumerate(sims), key=lambda item: -1 * item[1])
x = 0
for doc_position, doc_score in sims:
    print(doc_score, WOdata['Edited1'][doc_position])
    x += 1
    if (x > 20):
        break

0.6507647 ['3', 'abs', 'agitator', 'oil']
0.5617389 ['pm', 'inspect', 'abs', 'agitators']
0.53442883 ['u4', 'abs', 'inlet', 'duct', 'inspection']
0.5237368 ['4', '1', 'abs', 'building', 'sump', 'pump', 'repair']
0.5151138 ['2', '1', 'a', 'abs', 'recycle', 'pump', 'rebuild']
0.5137556 ['u4', 'abs', 'outlet', 'duct', 'inspection']
0.5057857 ['pm', '14', '2', '1', 'abs', 'vent', 'valve', 'lube']
0.5057857 ['pm', '14', '2', '1', 'abs', 'vent', 'valve', 'lube']
0.5036441 ['pm', '12', '2', '1', 'abs', 'tower', 'transmitter', 'calibration', 'pm']
0.5036441 ['pm', '12', '2', '1', 'abs', 'tower', 'transmitter', 'calibration', 'pm']
0.5036441 ['pm', '12', '2', '1', 'abs', 'tower', 'transmitter', 'calibration', 'pm']
0.50309664 ['pm', '14', '2', '3', 'abs', 'vent', 'valve', 'lube']
0.50309664 ['pm', '14', '2', '3', 'abs', 'vent', 'valve', 'lube']
0.50098884 ['pm', '12', '2', '3', 'abs', 'tower', 'transmitter', 'calibration', 'pm']
0.50098884 ['pm', '12', '2', '3', 'abs', 'tower', 'transmitter', '

In [ ]:
d2v = models.Doc2Vec('test')